In [2]:
from tqdm import tqdm
from datetime import datetime
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from urllib.parse import urljoin
import pandas as pd
import os
import warnings
import re

warnings.filterwarnings("ignore")

In [3]:
# 전체 건수 알아보기 (서울)

# 우회경로 설정
request_headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36')}


# 몇개씩 볼지
count_num = 10000

# link 설정
source = f'https://www.saramin.co.kr/zf_user/jobs/list/domestic?page=1&loc_mcd=101000&page_count={count_num}&isAjaxRequest=0&sort=RL&type=domestic&is_param=1&isSearchResultEmpty=1&isSectionHome=0&searchParamCount=1#searchTitle'

# 전체공고 확인
response = requests.get(url = source, headers =request_headers)
soup = BeautifulSoup(response.text, 'html.parser')
soup.select('#content > div.recruit_list_renew > div > div.area_title.list_total_count')

# 전체 공고 수
total_num = int((re.sub('[^0-9]','', soup.find_all("span", {"class":"total_count"})[0].get_text())))

# (전체 공고 / 10000) + 1 만큼 페이지 넘겨서 크롤링
page_nums = (total_num//count_num) + 1

In [5]:
print(page_nums)
page_nums=1

6


In [9]:
# 본격 크롤링 코드

# 전체공고 확인
response = requests.get(url = source, headers =request_headers)
soup = BeautifulSoup(response.text, 'html.parser')
soup.select('#content > div.recruit_list_renew')



# 채용공고 데이터 수집
job_data = []

# 채용공고 아이템들 찾기 (rec- 로 시작하는 id를 가진 div들)
job_items = soup.find_all('div', {'id': re.compile(r'^rec-\d+$')})

print(f"찾은 채용공고 개수: {len(job_items)}")

for page_num in range(1, page_nums+1) : 

    # link 설정
    source = f'https://www.saramin.co.kr/zf_user/jobs/list/domestic?page={page_num}&loc_mcd=101000&page_count={count_num}&isAjaxRequest=0&sort=RL&type=domestic&is_param=1&isSearchResultEmpty=1&isSectionHome=0&searchParamCount=1#searchTitle'

    for idx, item in enumerate(job_items, 1):
        try:
            job_id = item.get('id').replace('rec-', '')  # rec- 제거하고 숫자만 추출

            # 1. job_title과 job_link 추출
            title_link_element = item.find('a', {'id': f'rec_link_{job_id}'})
            job_title = title_link_element.get('title', '').strip() if title_link_element else ''
            job_link = urljoin('https://www.saramin.co.kr', title_link_element.get('href', '')) if title_link_element else ''

            # 2. comp_name 추출
            comp_name_element = item.select_one('div.box_item div.col.company_nm a')
            comp_name = comp_name_element.get_text(strip=True) if comp_name_element else ''

            # 3. job_sector 추출
            job_sector_element = item.select_one('div.box_item > div.col.notification_info > div.job_meta')
            if job_sector_element:
                # job_meta div 안의 모든 span 태그를 찾아 텍스트를 추출하여 리스트로 만듭니다.
                job_sector = [tag.get_text(strip=True) for tag in job_sector_element.find_all('span')]

                # '외'와 같이 불필요한 단어가 있다면 제거
                if job_sector and '외' in job_sector[-1]:
                    job_sector[-1] = job_sector[-1].replace('외', '')
            else:
                job_sector = []
            job_sector = job_sector[1:] # 자꾸 뭉친 게 나오네..

            # 4. work_place 추출
            work_place_element = item.select_one('div.box_item div.col.recruit_info ul li:nth-child(1) p')
            work_place = work_place_element.get_text(strip=True) if work_place_element else ''

            # 5. career 추출
            career_element = item.select_one('div.box_item div.col.recruit_info ul li:nth-child(2) p')
            career = career_element.get_text(strip=True) if career_element else ''

            # 6. education 추출
            education_element = item.select_one('div.box_item div.col.recruit_info ul li:nth-child(3) p')
            education = education_element.get_text(strip=True) if education_element else ''

            # 7. start_date 추출
            start_date_element = item.select_one('div.box_item div.col.support_info p span.date')
            start_date = start_date_element.get_text(strip=True) if start_date_element else ''

            # 8. deadline 추출
            deadline_element = item.select_one('div.box_item div.col.support_info p span.deadlines')
            deadline = deadline_element.get_text(strip=True) if deadline_element else ''


            # 데이터 저장
            job_info = {
                'job_id': job_id,
                'job_title': job_title,
                'job_link': job_link,
                'job_sector': job_sector,
                'comp_name': comp_name,
                'work_place': work_place,
                'career': career,
                'education': education,
                'start_date': start_date,
                'deadline': deadline
            }

            job_data.append(job_info)

            # 진행 상황 출력 (1000개마다)
            if idx % 1000 == 0:
                print(f"{page_num} 번째 페이지 진행 중... {idx}/{len(job_items)} 완료")

        except Exception as e:
            print(f"채용공고 {idx} 처리 중 오류 발생: {e}")
            continue              
            
            
    # 데이터 분할 저장 ==> 향후 DB 업로드 코드로 변경 필요
    time = datetime.now().strftime('%Y-%m-%d_%H%M%S')
    df = pd.DataFrame(job_data)
    df.to_excel(f'./data/사람인_서울_채용공고_{time}_{page_num}.xlsx', index=None)
    print(df.head())
    print()
    print()

찾은 채용공고 개수: 10000
1 번째 페이지 진행 중... 1000/10000 완료
1 번째 페이지 진행 중... 2000/10000 완료
1 번째 페이지 진행 중... 3000/10000 완료
1 번째 페이지 진행 중... 4000/10000 완료
1 번째 페이지 진행 중... 5000/10000 완료
1 번째 페이지 진행 중... 6000/10000 완료
1 번째 페이지 진행 중... 7000/10000 완료
1 번째 페이지 진행 중... 8000/10000 완료
1 번째 페이지 진행 중... 9000/10000 완료
1 번째 페이지 진행 중... 10000/10000 완료
     job_id                             job_title  \
0  51521787        [정규직/시청역]메리츠화재 유지관리 고객센터 상담 업무   
1  51345717    [정규직/신용산역] 차량관련 전반적인 단순 인바운드 상담 업무   
2  51441281  바로입사/주급 가능/FPCB/비제조/동반가능/주,야 교대근무 모집   
3  51510161         [강남구청/정규직] LG유플러스 고객센터 상담사 모집   
4  51521293         [강남구청] LG유플러스 MVNO 알뜰폰 상담사 모집   

                                            job_link  \
0  https://www.saramin.co.kr/zf_user/jobs/relay/v...   
1  https://www.saramin.co.kr/zf_user/jobs/relay/v...   
2  https://www.saramin.co.kr/zf_user/jobs/relay/v...   
3  https://www.saramin.co.kr/zf_user/jobs/relay/v...   
4  https://www.saramin.co.kr/zf_user/jobs/relay/v...   

               